In [2]:
pip install mysql-connector-python

   ---------------------------------------- 0.0/15.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.4 MB 660.6 kB/s eta 0:00:24
   ---------------------------------------- 0.1/15.4 MB 656.4 kB/s eta 0:00:24
   ---------------------------------------- 0.1/15.4 MB 1.1 MB/s eta 0:00:15
    --------------------------------------- 0.3/15.4 MB 1.8 MB/s eta 0:00:09
   - -------------------------------------- 0.5/15.4 MB 2.4 MB/s eta 0:00:07
   -- ------------------------------------- 0.8/15.4 MB 3.1 MB/s eta 0:00:05
   -- ------------------------------------- 1.1/15.4 MB 3.6 MB/s eta 0:00:04
   --- ------------------------------------ 1.4/15.4 MB 4.2 MB/s eta 0:00:04
   ---- ----------------------------------- 1.6/15.4 MB 4.6 MB/s eta 0:00:03
   ----- ---------------------------------- 1.9/15.4 MB 4.4 MB/s eta 0:00:04
   ----- ---------------------------------- 2.1/15.4 MB 4.3 MB/s eta 0:00:04
   ------ --------------------------------- 2.6/15.4 MB 4.8 MB/s eta 0:00:03
  

In [2]:
pip install pandas

     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------------------------- ------------ 41.0/61.0 kB 960.0 kB/s eta 0:00:01
     -------------------------------------- 61.0/61.0 kB 806.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
    --------------------------------------- 0.2/11.6 MB 6.1 MB/s eta 0:00:02
   - -------------------------------------- 0.4/11.6 MB 6.3 MB/s eta 0:00:02
   -- ------------------------------------- 0.7/11.6 MB 5.3 MB/s eta 0:00:03
   --- ------------------------------------ 1.0/11.6 MB 5.5 MB/s eta 0:00:02
   ----- ---------------------------------- 1.5/11.6 MB 6.2 MB/s eta 0:00:02
   ------ --------------------------------- 1.8/11.6 MB 6.4 MB/s eta 0:00:02
   ------- -------------------------------- 2.3/11.6 MB 6.9 MB/s eta 0:00:02
   --------- ------------------------------ 2.9/11.6 MB 7.6 MB/s eta 0:00:02
   ----------- ---------------------------- 3.3/11.6 MB 7.9 MB/s eta 0:00:02
   --

In [11]:
# creation au prealable de la base de donnée Projet_Fraude
#DEBUT import des bibliotheques nécessaires
import mysql.connector as msql
import pandas as pd
from mysql.connector import Error
#FIN import des bibliotheques nécessaires

In [12]:
#DEBUT variable
msg="Problème de connexion MySQL"
xpass = 'Bilal2010?'
xroot = 'root'
#FIN variable

#DEBUT Creation bdd Projet_Fraude
try:
    conn = msql.connect(host='localhost', user=xroot, password=xpass)
    if conn.is_connected():
        cursor = conn.cursor()
        #record = cursor.fetchone()
        print("Vous êtes  sur MYSQL")
        cursor.execute('DROP DATABASE IF EXISTS projet_fraudedb;')
        cursor.execute("CREATE DATABASE projet_fraudedb")
        print("la base projet_Fraude est créee")
    cursor.close
    conn.close     
except Error as e:
    print(msg, e)
#FIN Creation bdd Projet_Fraude
    




Vous êtes  sur MYSQL
la base projet_Fraude est créee


In [13]:
#DEBUT Creation structure des tables  Typepaiment et Transaction_Fraud
try:
    conn = msql.connect(host='localhost', database='projet_fraudedb', user=xroot, password=xpass)
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("Vous êtes connecté à la base: ", record)
       
        cursor.execute('DROP TABLE IF EXISTS credit_card_fraud_corr;')
        cursor.execute('DROP TABLE IF EXISTS transaction_fraude;') #suppression dans cette ordre car clef etrangere
        cursor.execute('DROP TABLE IF EXISTS typepaiment;')

        print('Création de la table credit_card_fraud_corr en cours....')
        cursor.execute("CREATE TABLE credit_card_fraud_corr (transactionID INT, step INT, type VARCHAR(50),amount FLOAT,NameOrig VARCHAR(150),oldbalanceOrg FLOAT,newbalanceOrig FLOAT,nameDest VARCHAR(150),oldbalanceDest FLOAT,newbalanceDest FLOAT,IsFraud INT)")
        print("La table credit_card_fraud_corr est crée....")

        print('Création de la table typepaiment en cours....') #creer la table typepaiment en premier car clef etrangere référencé dans transaction_fraude
        cursor.execute("CREATE TABLE typepaiment(typeID INT, Libelle VARCHAR(150), PRIMARY KEY(typeID))")
        print("La table typepaiment est crée....")

        print('Création de la table transaction_fraude en cours....')
        cursor.execute("CREATE TABLE transaction_fraude (transactionID INT,step INT,amount FLOAT,NameOrig VARCHAR(150),oldbalanceOrg FLOAT,newbalanceOrig FLOAT,nameDest VARCHAR(150),oldbalanceDest FLOAT,newbalanceDest FLOAT,IsFraud INT,typeID INT NOT NULL,PRIMARY KEY(transactionID),FOREIGN KEY(typeID) REFERENCES typepaiment(typeID))")
        print("La table transaction_fraude est crée....")

except Error as e:
    print(msg, e)
#FIN Creation structure des tables Typepaiment et Transaction_Fraud


#print("La base de données a été créée avec succès.")

Vous êtes connecté à la base:  ('projet_fraudedb',)
Création de la table credit_card_fraud_corr en cours....
La table credit_card_fraud_corr est crée....
Création de la table typepaiment en cours....
La table typepaiment est crée....
Création de la table transaction_fraude en cours....
La table transaction_fraude est crée....


In [14]:
#DEBUT chargement de credit_card_fraud_corr.csv dans la table transaction_fraude
#transactionData = pd.read_csv('credit_card_fraud_corr.csv',index_col=False,delimiter=';')
transactionData = pd.read_csv('credit_card_fraud_corr.csv',index_col=False,delimiter=';')
transactionData['amount'] = transactionData['amount'].replace(',', '.', regex=True).astype(float)
transactionData['oldbalanceOrg'] = transactionData['oldbalanceOrg'].replace(',', '.', regex=True).astype(float)
transactionData['newbalanceOrig'] = transactionData['newbalanceOrig'].replace(',', '.', regex=True).astype(float)
transactionData['oldbalanceDest'] = transactionData['oldbalanceDest'].replace(',', '.', regex=True).astype(float)
transactionData['newbalanceDest'] = transactionData['newbalanceDest'].replace(',', '.', regex=True).astype(float)
#transactionData = pd.read_xls('transaction_fraude.xlsx', index_col=False)
# Remplacer les lettres 'c' dans la colonne 'nameOrig' par '1'
transactionData['nameOrig'] = transactionData['nameOrig'].str.replace('C', '1')

# Remplacer les lettres 'c' dans la colonne 'nameDest' par '1'
transactionData['nameDest'] = transactionData['nameDest'].str.replace('C', '1')

# Remplacer les lettres 'm' dans la colonne 'nameDest' par '2'
transactionData['nameDest'] = transactionData['nameDest'].str.replace('M', '2')

# Afficher les premières lignes du DataFrame pour vérifier les modifications
transactionData.head(10)

,transactionId,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,0,1,PAYMENT,9839.64,11231006815,170136.00,160296.36,21979787155,0.0,0.00,0
1,1,1,PAYMENT,1864.28,11666544295,21249.00,19384.72,22044282225,0.0,0.00,0
2,2,1,TRANSFER,181.00,11305486145,181.00,0.00,1553264065,0.0,0.00,1
3,3,1,CASH_OUT,181.00,1840083671,181.00,0.00,138997010,21182.0,0.00,1
4,4,1,PAYMENT,11668.14,12048537720,41554.00,29885.86,21230701703,0.0,0.00,0
5,5,1,PAYMENT,7817.71,190045638,53860.00,46042.29,2573487274,0.0,0.00,0
6,6,1,PAYMENT,7107.77,1154988899,183195.00,176087.23,2408069119,0.0,0.00,0
7,7,1,PAYMENT,7861.64,11912850431,176087.23,168225.59,2633326333,0.0,0.00,0
8,8,1,PAYMENT,4024.36,11265012928,2671.00,0.00,21176932104,0.0,0.00,0
9,9,1,DEBIT,5337.77,1712410124,41720.00,36382.23,1195600860,41898.0,40348.79,0


In [6]:

try:
    conn = msql.connect(host='localhost', database='projet_fraudedb', user=xroot, password=xpass)
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("Vous êtes connecté à la base: ", record)


        for i,row in transactionData.iterrows():
            
          
            sql = "INSERT INTO projet_fraudedb.credit_card_fraud_corr VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row))
            conn.commit()
            
        print("les lignes de la dataframe (credit_card_fraud_corr.csv) ont été insérées dans la table credit_card_fraud_corr")

        cursor.execute("select * from projet_fraudedb.credit_card_fraud_corr limit 5")
        result=cursor.fetchall()
        print("Affichage des 5 premières lignes de la table credit_card_fraud_corr: ")
        for i in result:
            print(result)
        # Alimenter la table Typepaiement en sql (manuelllement) :

        cursor.execute("INSERT INTO typepaiement (typeID, Libelle) VALUES (1 , 'PAYMENT')")
        cursor.execute("INSERT INTO typepaiement (typeID, Libelle) VALUES (2 , 'TRANSFER')")
        cursor.execute("INSERT INTO typepaiement (typeID, Libelle) VALUES (3 , 'CASH_OUT')")
        cursor.execute("INSERT INTO typepaiement (typeID, Libelle) VALUES (4 , 'CASH_IN')")
        cursor.execute("INSERT INTO typepaiement (typeID, Libelle) VALUES (5 , 'DEBIT')")
        
        

        cursor.close
        conn.close       CASH_IN

except Error as e:
    print(msg, e)
#FIN chargement de credit_card_fraud_corr.csv dans la table transaction_fraude

Vous êtes connecté à la base:  ('projet_fraudedb',)
les lignes de la dataframe (credit_card_fraud_corr.csv) ont été insérées dans la table credit_card_fraud_corr
Affichage des 5 premières lignes de la table credit_card_fraud_corr: 
[(0, 1, 'PAYMENT', 9839.64, 'C1231006815', 170136.0, 160296.0, 'M1979787155', 0.0, 0.0, 0), (1, 1, 'PAYMENT', 1864.28, 'C1666544295', 21249.0, 19384.7, 'M2044282225', 0.0, 0.0, 0), (2, 1, 'TRANSFER', 181.0, 'C1305486145', 181.0, 0.0, 'C553264065', 0.0, 0.0, 1), (3, 1, 'CASH_OUT', 181.0, 'C840083671', 181.0, 0.0, 'C38997010', 21182.0, 0.0, 1), (4, 1, 'PAYMENT', 11668.1, 'C2048537720', 41554.0, 29885.9, 'M1230701703', 0.0, 0.0, 0)]
[(0, 1, 'PAYMENT', 9839.64, 'C1231006815', 170136.0, 160296.0, 'M1979787155', 0.0, 0.0, 0), (1, 1, 'PAYMENT', 1864.28, 'C1666544295', 21249.0, 19384.7, 'M2044282225', 0.0, 0.0, 0), (2, 1, 'TRANSFER', 181.0, 'C1305486145', 181.0, 0.0, 'C553264065', 0.0, 0.0, 1), (3, 1, 'CASH_OUT', 181.0, 'C840083671', 181.0, 0.0, 'C38997010', 21182.0

In [20]:

try:
    conn = msql.connect(host='localhost', database='projet_fraudedb', user=xroot, password=xpass)
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("Vous êtes connecté à la base: ", record)
        cursor.execute("delete from typepaiment")
        cursor.execute("INSERT INTO typepaiment (typeID, Libelle) VALUES (1 , 'PAYMENT')")
        cursor.execute("INSERT INTO typepaiment (typeID, Libelle) VALUES (2 , 'TRANSFER')")
        cursor.execute("INSERT INTO typepaiment (typeID, Libelle) VALUES (3 , 'CASH_OUT')")
        cursor.execute("INSERT INTO typepaiment (typeID, Libelle) VALUES (4 , 'CASH_IN')")
        cursor.execute("INSERT INTO typepaiment (typeID, Libelle) VALUES (5 , 'DEBIT')")
        
        

        cursor.close
        conn.close       

except Error as e:
    print(msg, e)
#FIN chargement de credit_card_fraud_corr.csv dans la table transaction_fraude

Vous êtes connecté à la base:  ('projet_fraudedb',)
